# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 71 new papers today
          43 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/42 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.06829


extracting tarball to tmp_2209.06829...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.06830


extracting tarball to tmp_2209.06830...

...

/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.07395/ISW.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'texbase/macros' from 'tmp_2209.07395/texbase/macros.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2209.07412...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.07460


extracting tarball to tmp_2209.07460...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06833-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06833) | **A Sun-like star orbiting a black hole**  |
|| Kareem El-Badry, et al. -- incl., <mark>Hans-Walter Rix</mark>, <mark>Rhys Seeburger</mark>, <mark>Silvia Almada Monter</mark>, <mark>Jennifer Wojno</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| *29 pages, 15 figures, 1-2 black holes. Submitted to MNRAS*|
|**Abstract**| We report discovery of a bright, nearby ($G = 13.8$; $d = 480$ pc) Sun-like star orbiting a dark object. We identified the system as a black hole candidate via its astrometric orbital solution from the Gaia mission. Radial velocity monitoring validated and refined the Gaia solution, and spectroscopy ruled out significant light contributions from another star. Joint modeling of radial velocities and astrometry constrains the companion mass to $M_2 = 9.8\pm 0.2\,M_{\odot}$. The spectroscopic orbit alone sets a minimum companion mass of $M_2>5\,M_{\odot}$; if the companion were a $5\,M_{\odot}$ star, it would be 500 times more luminous than the entire system. These constraints are insensitive to the assumed mass of the luminous star, which appears as a slowly-rotating G dwarf ($T_{\rm eff}=5850$ K, $\log g = 4.5$, $M=0.93\,M_{\odot}$), with near-solar metallicity ([Fe/H] = -0.2) and an unremarkable abundance pattern. We find no plausible astrophysical scenario that can explain the orbit and does not involve a black hole. The orbital period, $P_{\rm orb}=185.6$ days, is longer than that of any known stellar-mass black hole binary, and the eccentricity is modest, $e=0.45$. The system's Galactic orbit is typical of thin-disk stars, suggesting that it formed in the Milky Way disk with at most a weak natal kick. Explaining the system's formation with standard binary evolutionary models is challenging: it is difficult for the luminous star to survive a common envelope event under standard assumptions, and difficult for it to end up in a wide orbit afterward. Formation models involving triples or dynamical assembly in an open cluster may be more promising. This is the nearest known black hole by a factor of 3, and its discovery suggests the existence of a sizable population of dormant black holes in binaries. Future Gaia releases will likely facilitate the discovery of dozens more. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07339-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07339) | **A-SLOTH: Ancient Stars and Local Observables by Tracing Halos**  |
|| <mark>Mattis Magg</mark>, Tilman Hartwig, <mark>Li-Hsin Chen</mark>, Yuta Tarumi |
|*Appeared on*| *2022-09-16*|
|*Comments*| *Published in JOSS. Git repo is available at this https URL*|
|**Abstract**| Galaxies are thought to reside inside of large gravitationally bound structures of dark matter, so-called haloes. While the smallest of these haloes host no or only a few stars, the biggest host entire clusters of galaxies. Over cosmic history, haloes often collided and merged, forming bigger and bigger structures. Merger trees, i.e., catalogues of haloes evolving and connections between them as they grow and merge, have become a vital tool in describing and understanding the history of cosmological objects such as our Galaxy. Semi-analytical models, built on top of such merger trees, are a common approach for theoretical studies in cosmology. The semi-analytical nature of such models is especially beneficial when the dynamic range in spatial and time scales that need to be considered becomes too large for numerical simulations. Ancient Stars and Local Observables by Tracing Halos (A-SLOTH) is such a semi-analytical model and it is designed to simulate star formation in the early Universe in a fast and accessible way. It uses merger trees, either from numerical simulations or generated by statistical algorithms to describe the history of galaxies. The processes of baryonic physics, in particular gas cooling, star formation and stellar feedback are described with approximations and statistical models. The range of applications for this model is extensive and we, therefore, make it available to the scientific community. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06829-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06829) | **A Joint Roman Space Telescope and Rubin Observatory Synthetic Wide-Field  Imaging Survey**  |
|| M. A. Troxel, et al. -- incl., <mark>C. Lin</mark>, <mark>C. W. Walter</mark>, <mark>T. Zhang</mark>, <mark>A. Hearin</mark>, <mark>K. Heitmann</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| **|
|**Abstract**| We present and validate 20 deg$^2$ of overlapping synthetic imaging surveys representing the full depth of the Nancy Grace Roman Space Telescope High-Latitude Imaging Survey (HLIS) and five years of observations of the Vera C. Rubin Observatory Legacy Survey of Space and Time (LSST). The two synthetic surveys are summarized, with reference to the existing 300 deg$^2$ of LSST simulated imaging produced as part of Dark Energy Science Collaboration (DESC) Data Challenge 2 (DC2). Both synthetic surveys observe the same simulated DESC DC2 universe. For the synthetic Roman survey, we simulate for the first time fully chromatic images along with the detailed physics of the Sensor Chip Assemblies derived from lab measurements using the flight detectors. The simulated imaging and resulting pixel-level measurements of photometric properties of objects span a wavelength range of $\sim$0.3 to 2.0 $\mu$m. We also describe updates to the Roman simulation pipeline, changes in how astrophysical objects are simulated relative to the original DC2 simulations, and the resulting simulated Roman data products. We use these simulations to explore the relative fraction of unrecognized blends in LSST images, finding that 20-30% of objects identified in LSST images with $i$-band magnitudes brighter than 25 can be identified as multiple objects in Roman images. These simulations provide a unique testing ground for the development and validation of joint pixel-level analysis techniques of ground- and space-based imaging data sets in the second half of the 2020s -- in particular the case of joint Roman--LSST analyses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07460-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07460) | **Multi-probe analysis of the galaxy cluster CL J1226.9+3332: Hydrostatic  mass and hydrostatic-to-lensing bias**  |
|| M. Muñoz-Echeverría, et al. -- incl., <mark>C. Kramer</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| **|
|**Abstract**| The precise estimation of the mass of galaxy clusters is a major issue for cosmology. Large galaxy cluster surveys rely on scaling laws that relate cluster observables to their masses. From the high resolution observations of ~ 45 galaxy clusters with NIKA2 and XMM-Newton instruments, the NIKA2 SZ Large Program should provide an accurate scaling relation between the thermal Sunyaev-Zel'dovich effect and the hydrostatic mass. In this paper, we present an exhaustive analysis of the hydrostatic mass of the well known galaxy cluster CL J1226.9+3332, the highest-redshift cluster in the NIKA2 SZ Large Program at z = 0.89. We combine the NIKA2 observations with thermal Sunyaev-Zel'dovich data from NIKA, Bolocam and MUSTANG instruments and XMM-Newton X-ray observations and test the impact of the systematic effects on the mass reconstruction. We conclude that slight differences in the shape of the mass profile can be crucial when defining the integrated mass at R500, which demonstrates the importance of the modeling in the mass determination. We prove the robustness of our hydrostatic mass estimates by showing the agreement with all the results found in the literature. Another key information for cosmology is the bias of the masses estimated assuming hydrostatic equilibrium hypothesis. Based on the lensing convergence maps from the Cluster Lensing And Supernova survey with Hubble (CLASH) data, we obtain the lensing mass estimate for CL J1226.9+3332. From this we are able to measure the hydrostatic-to-lensing mass bias for this cluster, that spans from 1 - bHSE/lens ~ 0.7 to 1, presenting the impact of data-sets and mass reconstruction models on the bias. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07294-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07294) | **CUBES and its software ecosystem: instrument simulation, control, and  data processing**  |
|| Giorgio Calderone, et al. -- incl., <mark>Florian Rothmaier</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| *SPIE proceedings, SPIE Astronomical Telescopes + Instrumentation 2022, Montr\'eal, Canada*|
|**Abstract**| CUBES (Cassegrain U-Band Efficient Spectrograph) is the recently approved high-efficiency VLT spectrograph aimed to observe the sky in the UV ground-based region (305-400 nm) with a high-resolution mode (~20K) and a low-resolution mode (~5K). In this paper we will briefly describe the requirements and the design of the several software packages involved in the project, namely the instrument control software, the exposure time calculator, the end-to-end simulator, and the data reduction software suite. We will discuss how the above mentioned blocks cooperate to build up a "software ecosystem" for the CUBES instrument, and to support the users from the proposal preparation to the science-grade data products. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07182-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07182) | **SOXS mechanical integration and verification in Italy**  |
|| M. Aliverti, et al. -- incl., <mark>O. Hershko</mark>, <mark>M. Hernandez</mark>, <mark>G . Li Causi</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| *arXiv admin note: text overlap with arXiv:2012.12693*|
|**Abstract**| SOXS (SOn of X-Shooter) is a medium resolution (~4500) wide-band (0.35 - 2.0 {\mu}m) spectrograph which passed the Final Design Review in 2018. The instrument is in the final integration phase and it is planned to be installed at the NTT in La Silla by next year. It is mainly composed of five different optomechanical subsystems (Common Path, NIR spectrograph, UV-VIS spectrograph, Camera, and Calibration) and other mechanical subsystems (Interface flange, Platform, cable corotator, and cooling system). A brief overview of the optomechanical subsystems is presented here as more details can be found in the specific proceedings while a more comprehensive discussion is dedicated to the other mechanical subsystems and the tools needed for the integration of the instrument. Moreover, the results obtained during the acceptance of the various mechanical elements are presented together with the experiments performed to validate the functionality of the subsystems. Finally, the mechanical integration procedure is shown here, along with all the modifications applied to correct the typical problems happening in this phase. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07201-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07201) | **Progress on the SOXS NIR Spectrograph AIT**  |
|| Fabrizio Vitali, et al. -- incl., <mark>Marcos Hernandez Diaz</mark>, <mark>Ofir Hershko</mark>, <mark>Gianluca Li Causi</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| *6 pages, 8 Figures, SPIE: Astronomical Telescope + Instrumentation, Montreal 2022*|
|**Abstract**| The Son Of X-Shooter (SOXS) is a single object spectrograph, built by an international consortium for the 3.58-m ESO New Technology Telescope at the La Silla Observatory, ranging from 350 to 2000 nm. In this paper, we present the progress in the AIT phase of the Near InfraRed (NIR) arm. We describe the different AIT phases of the cryo, vacuum, opto-mechanics and detector subsystems, that finally converged at the INAF-OAB premises in Merate (Italy), where the NIR spectrograph is currently being assembled and tested, before the final assembly on SOXS. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.07208-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.07208) | **The vacuum and cryogenics system of the SOXS spectrograph**  |
|| S. Scuderi, et al. -- incl., <mark>O. Hershko</mark>, <mark>M. Hernandez Diaz</mark>, <mark>G. Li Causi</mark> |
|*Appeared on*| *2022-09-16*|
|*Comments*| *12 pages, 6 figures, SPIE proceedings of the conference Advances in Optical and Mechanical Technologies for Telescopes and Instrumentation IV*|
|**Abstract**| SOXS (Son Of X-Shooter) is a single object spectrograph built by an international consortium for the ESO NTT telescope. SOXS is based on the heritage of the X-Shooter at the ESO-VLT with two arms (UV-VIS and NIR) working in parallel, with a Resolution-Slit product of about 4500, capable of simultaneously observing over the entire band the complete spectral range from the U- to the H-band. SOXS will carry out rapid and long-term Target of Opportunity requests on a variety of astronomical objects. The SOXS vacuum and cryogenic control system has been designed to evacuate, cool down and maintain the UV-VIS detector and the entire NIR spectrograph to their operating temperatures. The design chosen allows the two arms to be operated independently. This paper describes the final design of the cryo-vacuum control system, its functionalities and the tests performed in the integration laboratories. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.06833.md
    + _build/html/tmp_2209.06833/./gaia_bh1_mosaic.png
    + _build/html/tmp_2209.06833/./gaia_bh1_rvfig.png
    + _build/html/tmp_2209.06833/./gaia_bh1_search_volume_different_cuts.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\shrug}[1][]{$
$\begin{tikzpicture}[baseline,x=0.8\ht\strutbox,y=0.8\ht\strutbox,line width=0.125ex,#1]$
$\newcommand{\arm}{(-2.5,0.95) to (-2,0.95) (-1.9,1) to (-1.5,0) (-1.35,0) to (-0.8,0)}$
$\draw \arm;$
$\draw[xscale=-1] \arm;$
$\newcommand{\headpart}{(0.6,0) arc[start angle=-40, end angle=40,x radius=0.6,y radius=0.8]}$
$\draw \headpart;$
$\draw[xscale=-1] \headpart;$
$\newcommand{\eye}{(-0.075,0.15) .. controls (0.02,0) .. (0.075,-0.15)}$
$\draw[shift={(-0.3,0.8)}] \eye;$
$\draw[shift={(0,0.85)}] \eye;$
$\draw (-0.1,0.2) to [out=15,in=-100] (0.4,0.95);$
$\end{tikzpicture}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\xmark}{\ding{55}}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\arm}{(-2.5,0.95) to (-2,0.95) (-1.9,1) to (-1.5,0) (-1.35,0) to (-0.8,0)}$
$\newcommand{\headpart}{(0.6,0) arc[start angle=-40, end angle=40,x radius=0.6,y radius=0.8]}$
$\newcommand{\eye}{(-0.075,0.15) .. controls (0.02,0) .. (0.075,-0.15)}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{\shrug}[1][]{$
$\begin{tikzpicture}[baseline,x=0.8\ht\strutbox,y=0.8\ht\strutbox,line width=0.125ex,#1]$
$\newcommand{$\arm$}{(-2.5,0.95) to (-2,0.95) (-1.9,1) to (-1.5,0) (-1.35,0) to (-0.8,0)}$
$\draw $\arm$;$
$\draw[xscale=-1] $\arm$;$
$\newcommand{$\headpart$}{(0.6,0) arc[start angle=-40, end angle=40,x radius=0.6,y radius=0.8]}$
$\draw $\headpart$;$
$\draw[xscale=-1] $\headpart$;$
$\newcommand{$\eye$}{(-0.075,0.15) .. controls (0.02,0) .. (0.075,-0.15)}$
$\draw[shift={(-0.3,0.8)}] $\eye$;$
$\draw[shift={(0,0.85)}] $\eye$;$
$\draw (-0.1,0.2) to [out=15,in=-100] (0.4,0.95);$
$\end{tikzpicture}}$
$\newcommand{$\red$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\blue$}[1]{\textcolor{blue}{#1}}$
$\newcommand{$\xmark$}{\ding{55}}$
$\newcommand{$\cmark$}{\ding{51}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\arm$}{(-2.5,0.95) to (-2,0.95) (-1.9,1) to (-1.5,0) (-1.35,0) to (-0.8,0)}$
$\newcommand{$\headpart$}{(0.6,0) arc[start angle=-40, end angle=40,x radius=0.6,y radius=0.8]}$
$\newcommand{$\eye$}{(-0.075,0.15) .. controls (0.02,0) .. (0.075,-0.15)}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# A Sun-like star orbiting a black hole

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.06833-b31b1b.svg)](https://arxiv.org/abs/2209.06833)<mark>Appeared on: 2022-09-16</mark> - _29 pages, 15 figures, 1-2 black holes. Submitted to MNRAS_

</div>
<div id="authors">

Kareem El-Badry, et al. -- incl., <mark><mark>Hans-Walter Rix</mark></mark>, <mark><mark>Rhys Seeburger</mark></mark>, <mark><mark>Silvia Almada Monter</mark></mark>, <mark><mark>Jennifer Wojno</mark></mark>

</div>
<div id="abstract">

**Abstract:** We report  discovery of a bright, nearby ($G = 13.8;  d = 480 \rm pc$) Sun-like star orbiting a dark object. We identified the system as a black hole candidate via its astrometric orbital solution from the{\it Gaia}mission. Radial velocity monitoring validated and refined the{\it Gaia}solution, and spectroscopy ruled out significant light contributions from another star. Joint modeling of radial velocities and astrometry constrains the companion mass to$M_2 = 9.8\pm 0.2 M_{\odot}$. The spectroscopic orbit alone sets a minimum companion mass of$M_2>5 M_{\odot}$; if the companion were a$5 M_{\odot}$star, it would be$500$times more luminous than the entire system. These constraints are  insensitive to the assumed mass of the luminous star, which appears as a slowly-rotating G dwarf ($T_{\rm eff}=5850 \rm K$,$\log g = 4.5$,$M=0.93 M_{\odot}$), with near-solar metallicity ($\rm[Fe/H] = -0.2$) and an unremarkable abundance pattern. We find no plausible astrophysical scenario that can explain the orbit and does not involve a black hole. The orbital period,$P_{\rm orb}=185.6$days, is longer than that of any known stellar-mass black hole binary, and the eccentricity is modest,$e=0.45$. The system's Galactic orbit is typical of thin-disk stars, suggesting that it formed in the Milky Way disk with at most a weak natal kick. Explaining the system's formation with standard binary evolutionary models is challenging: it is difficult for the luminous star to survive a common envelope event under standard assumptions, and difficult for it to end up in a wide orbit afterward. Formation models involving triples or dynamical assembly in an open cluster may be more promising.This is the nearest known black hole by a factor of 3, and its discovery suggests the existence of a sizable population of dormant black holes in binaries. Future{\it Gaia}releases will likely facilitate the discovery of dozens more.

</div>

<div id="div_fig1">

<img src="tmp_2209.06833/./gaia_bh1_mosaic.png" alt="Fig2" width="100%"/>

**Figure 2. -** Properties of the luminous star. Upper left shows a 45 arcsecond wide $z$/$g$ PanSTARRS postage stamp centered on the source. Upper right shows the broadband SED and a single-star fit. Red points show observed photometry; open black squares show the integrated fluxes predicted for the model spectrum (black line). Lower left compares the source to the rest of the {\it Gaia} astrometric binary sample on the color-magnitude diagram. Lower right compares the temperature and radius measured from the SED to single-star MIST evolutionary models with [Fe/H] = -0.2 (as measured from spectroscopy), which suggest a mass of $M_\star \approx 0.93 M_{\odot}$.  (*fig:mosaic*)

</div>
<div id="div_fig2">

<img src="tmp_2209.06833/./gaia_bh1_rvfig.png" alt="Fig3" width="100%"/>

**Figure 3. -** Radial velocities. Points with error bars are measurements; gray lines are draws from the posterior when jointly fitting these RVs and the {\it Gaia} astrometric constraints. Top panel shows all available RVs, including observations by the LAMOST survey in 2017 and 2019; bottom panel highlights our follow-up in 2022. The best-fit solution has a period of 186 days, eccentricity 0.45, and RV semi-amplitude of $68 \rm km s^{-1}$. Together with the inclination constraint from astrometry, this implies a companion mass of $9.8\pm 0.2 M_{\odot}$.  (*fig:rvfig*)

</div>
<div id="div_fig3">

<img src="tmp_2209.06833/./gaia_bh1_search_volume_different_cuts.png" alt="Fig1" width="100%"/>

**Figure 1. -** Number of sources in {\it Gaia} DR3 in which a $10 M_{\odot}$ dark companion could be detected. Black line shows the cuts employed in DR3 (Equations \ref{eq:cut} and \ref{eq:acut}), which remove most short-period binaries. Dashed red line corresponds to simply requiring $a_0/\sigma_{a_0} > 10$ and $\varpi/\sigma_{\varpi} > 10$; this results in a much larger searchable volume at short periods. Binaries with periods near 1 year are excluded, being difficult to detect with astrometry.   (*fig:volume_different_cuts*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

33  publications in the last 7 days.
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers